# Impor packages

In [1]:
# pip install pandas
# pip install numpy
# pip install datetime
import pandas as pd
import numpy as np
import datetime as dt

# Impor data dari CSV ke DataFrame

In [2]:
df = pd.read_csv('Online Retail Data.csv', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5.0,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1.0,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5.0,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1.0,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1.0,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
148686,509208,71459,HANGING JAM JAR T-LIGHT HOLDER,12.0,2010-05-21 09:40:00,0.85,NaN
148687,509208,21612,SET 12 WILDFLOWER BOTANICAL T-LIGHT,2.0,2010-05-21 09:40:00,2.95,NaN
148688,509208,21614,SET OF 12 ROSE BOTANICAL T-LIGHTS,2.0,2010-05-21 09:40:00,2.95,NaN
148689,509208,21609,SET 12 LAVENDER BOTANICAL T-LIGHTS,2.0,2010-05-21 09:40:00,2.95,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148691 entries, 0 to 148690
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      148691 non-null  object 
 1   product_code  148691 non-null  object 
 2   product_name  147276 non-null  object 
 3   quantity      148690 non-null  float64
 4   order_date    148690 non-null  object 
 5   price         148690 non-null  float64
 6   customer_id   116070 non-null  float64
dtypes: float64(3), object(4)
memory usage: 7.9+ MB


# Data cleansing

In [5]:
df_clean = df.copy()
# membuat kolom date
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date
df_clean['date'] = pd.to_datetime(df_clean['date'])
# menghapus semua baris tanpa customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
# menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()
# menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# menghapus baris dengan status cancelled, yaitu yang order_id-nya diawali 'C'
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount
0,493414,21844,retro spot mug,36.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,91.80
1,493414,21533,retro spot large milk jug,12.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,51.00
2,493414,37508,new england ceramic cake server,2.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,5.10
3,493414,35001G,hand open shape gold,2.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,8.50
4,493414,21527,retro spot traditional teapot,12.0,2010-01-04 10:28:00,6.95,14590.0,2010-01-04,83.40
...,...,...,...,...,...,...,...,...,...
112010,509202,22138,baking set 9 piece retrospot,3.0,2010-05-21 09:22:00,4.95,17738.0,2010-05-21,14.85
112011,509204,79320,flamingo lights,48.0,2010-05-21 09:27:00,4.25,14258.0,2010-05-21,204.00
112012,509204,37446,mini cake stand with hanging cakes,32.0,2010-05-21 09:27:00,1.25,14258.0,2010-05-21,40.00
112013,509204,71477,colour glass. star t-light holder,24.0,2010-05-21 09:27:00,2.75,14258.0,2010-05-21,66.00


In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112015 entries, 0 to 112014
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      112015 non-null  object        
 1   product_code  112015 non-null  object        
 2   product_name  112015 non-null  object        
 3   quantity      112015 non-null  float64       
 4   order_date    112015 non-null  object        
 5   price         112015 non-null  float64       
 6   customer_id   112015 non-null  object        
 7   date          112015 non-null  datetime64[ns]
 8   amount        112015 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 7.7+ MB


# Menyiapkan data basket

## Buat DataFrame basket

In [7]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,12 pencils small tube red spotty,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
509200,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
509201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5218 entries, 493414 to 509204
Columns: 3206 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3206)
memory usage: 127.7+ MB


## Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0

In [9]:
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.applymap(encode)
basket_encode

<ipython-input-9-31958d44448d>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encode = basket.applymap(encode)


product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,12 pencils small tube red spotty,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509197,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
509200,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
509201,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5218 entries, 493414 to 509204
Columns: 3206 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3206)
memory usage: 16.0+ MB


## Ambil transaksi dengan banyaknya produk unik lebih dari 1 saja

In [11]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,12 pencils small tube red spotty,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509196,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
509197,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
509200,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4859 entries, 493414 to 509204
Columns: 3206 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3206)
memory usage: 14.9+ MB


# Mengaplikasikan apriori algorithm

## Buat list frequent itemset (kumpulan produk yang sering dibeli)

In [28]:
pip install mlxtend

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,product_cnt
0,0.201070,(white hanging heart t-light holder),1
1,0.098580,(jumbo bag red white spotty),1
2,0.097345,(door mat union flag),1
3,0.089319,(pack of 72 retro spot cake cases),1
4,0.083968,(home building block word),1
...,...,...,...
1362,0.010084,"(wooden frame antique white, cream cupid heart...",2
1363,0.010084,"(retro spot cake stand, retro spot storage jar)",2
1364,0.010084,"(jumbo storage bag skulls, jumbo bag strawberry)",2
1365,0.010084,"(pack of 72 retro spot cake cases, jumbo bag s...",2


## Hitung nilai support, confidence, dan lift dari setiap pasangan produk yang mungkin

In [31]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7,  num_itemsets=len(frequent_itemset)).sort_values(['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.055979,0.201070,0.044454,0.794118,3.949455,1.0,0.033198,3.880516,0.791084,0.209100,0.742302,0.507601
1,(toilet metal sign),(bathroom metal sign),0.033752,0.047952,0.027783,0.823171,17.166466,1.0,0.026165,5.383994,0.974643,0.515267,0.814264,0.701285
2,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.036221,0.079852,0.025725,0.710227,8.894315,1.0,0.022833,3.175413,0.920926,0.284738,0.685080,0.516196
3,(blue 3 piece mini dots cutlery set),(red 3 piece mini dots cutlery set),0.035398,0.037456,0.025314,0.715116,19.092033,1.0,0.023988,3.378725,0.982397,0.532468,0.704030,0.695470
4,(wooden picture frame white finish),(wooden frame antique white),0.034163,0.064622,0.025108,0.734940,11.372842,1.0,0.022900,3.528925,0.944333,0.340782,0.716628,0.561737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,(green spotty cup),(red spotty cup),0.013171,0.023667,0.010084,0.765625,32.349321,1.0,0.009773,4.165686,0.982022,0.376923,0.759943,0.595856
239,"(pack of 60 pink paisley cake cases, pack of 6...",(pack of 72 retro spot cake cases),0.013377,0.089319,0.010084,0.753846,8.439950,1.0,0.008890,3.699642,0.893468,0.108889,0.729704,0.433375
240,"(pack of 60 mushroom cake cases, 60 teatime fa...",(pack of 72 retro spot cake cases),0.013583,0.089319,0.010084,0.742424,8.312072,1.0,0.008871,3.535586,0.891807,0.108647,0.717161,0.427664
241,"(jumbo bag pink with white spots, jumbo bag sc...",(jumbo bag red white spotty),0.013583,0.098580,0.010084,0.742424,7.531189,1.0,0.008745,3.499631,0.879161,0.098790,0.714256,0.422360
